# Envío de mensajes al DMD desde macOS

> Este notebook instala `pyserial`, detecta el puerto en macOS y envía un mensaje al sketch `kroner_display_dmd.ino`.

- El protocolo es `XXYYT F PP TEXT...` (p. ej. `00004:00HOLA`).
- Usa 9600 baudios, sin terminador de línea.
- Si no detecta puerto automáticamente, fija la variable `PORT` manualmente (p. ej. `/dev/tty.usbserial-1410`).

In [1]:
# Asegura pyserial y elimina el paquete conflictivo 'serial'
import sys, subprocess

# Desinstala el paquete 'serial' (no pyserial) si estuviera instalado y crea conflicto
try:
    subprocess.call([sys.executable, "-m", "pip", "uninstall", "-y", "serial"])  # paquete equivocado
except Exception:
    pass

# Instala/actualiza pyserial
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pyserial"])  # paquete correcto

import serial
print("pyserial importado, tiene Serial:", hasattr(serial, "Serial"))

pyserial importado, tiene Serial: True



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [66]:
# Detección de puerto en macOS y envío del mensaje al Arduino
from serial.tools import list_ports
import serial, time

# Lista puertos disponibles y sugiere candidatos típicos en macOS
ports = list(list_ports.comports())
print("Puertos detectados:")
for p in ports:
    print(" -", p.device, "|", p.description)

# Ajusta aquí manualmente si lo conoces (por ejemplo: "/dev/tty.usbserial-1410" o "/dev/tty.usbmodem1101")
PORT = None

# Intento automático: elegir el primer puerto que contenga "usb"
if PORT is None:
    candidates = [p.device for p in ports if "usb" in p.device.lower()]
    PORT = candidates[0] if candidates else None

if not PORT:
    raise RuntimeError("No se encontró un puerto serie. Conecta el dispositivo y vuelve a ejecutar la celda.")

print("Usando puerto:", PORT)

# Abre el puerto y envía el mensaje con el protocolo del sketch
ser = serial.Serial(PORT, 115200, timeout=2)
try:
    # Espera un momento para que se establezca la conexión
    time.sleep(2)
    
    # Limpia el buffer de entrada por si hay datos previos
    ser.reset_input_buffer()
    
    # Envía el mensaje
    msg = "RD"
    ser.write(msg.encode("latin-1"))
    ser.flush()
    print("Mensaje enviado:", msg)
    
    # Espera y lee la respuesta
    time.sleep(1)  # Da tiempo al dispositivo para responder
    
    # Lee la respuesta disponible
    if ser.in_waiting > 0:
        respuesta = ser.read(ser.in_waiting).decode("latin-1", errors="ignore")
        print("\n--- Respuesta recibida ---")
        print(respuesta)
        print("--- Fin de respuesta ---")
    else:
        print("\nNo se recibió respuesta del dispositivo.")
        
finally:
    ser.close()


Puertos detectados:
 - /dev/cu.debug-console | n/a
 - /dev/cu.TelmoMM | n/a
 - /dev/cu.Bluetooth-Incoming-Port | n/a
 - /dev/cu.usbserial-010DC655 | CP2104 USB to UART Bridge Controller
 - /dev/cu.SLAB_USBtoUART7 | CP2104 USB to UART Bridge Controller
 - /dev/cu.usbserial-0001 | CP2102 USB to UART Bridge Controller
 - /dev/cu.SLAB_USBtoUART9 | CP2102 USB to UART Bridge Controller
 - /dev/cu.usbserial-01657030 | CP2104 USB to UART Bridge Controller
 - /dev/cu.SLAB_USBtoUART | CP2104 USB to UART Bridge Controller
Usando puerto: /dev/cu.usbserial-010DC655
Mensaje enviado: RD

No se recibió respuesta del dispositivo.


In [4]:
import serial
import time

DEVICE = "/dev/cu.usbserial-010DC655"
BAUDRATES = [9600, 19200, 38400, 115200]

for baud in BAUDRATES:
    try:
        print(f"Probando baudrate: {baud}")
        ser = serial.Serial(DEVICE, baudrate=baud, timeout=1)
        ser.write(b'AT\r\n')
        time.sleep(0.5)
        response = ser.read(100)  # leemos hasta 100 bytes
        print("Respuesta:", response.decode(errors='ignore'))
        ser.close()
    except Exception as e:
        print("Error:", e)
    print("="*30)


Probando baudrate: 9600
Respuesta: 
Probando baudrate: 19200
Respuesta: 
Probando baudrate: 38400
Respuesta: 
Probando baudrate: 115200
Respuesta: 


In [9]:
import serial
s = serial.Serial(DEVICE, 9600)
s.setRTS(True)
s.setDTR(False)
s.write(b'AT\r\n')



4

In [11]:
import serial, time

s = serial.Serial(DEVICE, 9600, timeout=1)

# 1) Bajar SET → modo AT
s.setRTS(True)    # RTS activo = SET LOW

# 2) Apagar módulo
s.setDTR(True)    # DTR activo = EN LOW → módulo apagado
time.sleep(0.2)

# 3) Encender módulo con SET ya low
s.setDTR(False)   # DTR inactivo = EN HIGH → módulo encendido
time.sleep(0.5)

# 4) Enviar comando AT
s.write(b'AT\r\n')
print("Respuesta:", s.readline())


Respuesta: b''


In [12]:
import serial, time

#DEVICE = "/dev/cu.usbserial-01657030"
BAUD = 9600

ser = serial.Serial(DEVICE, baudrate=BAUD, timeout=0.5)  # timeout razonable
# Opcional: forzar líneas de control (ajusta True/False según necesites)
ser.setRTS(False)
ser.setDTR(True)
time.sleep(0.05)

# limpiar buffers
ser.reset_input_buffer()
ser.reset_output_buffer()

# enviar AT
ser.write(b'AT\r\n')
ser.flush()

# esperar un poco para la respuesta
time.sleep(0.2)

# leer todo lo disponible
resp = ser.read_all()        # lee todo lo que haya en el buffer
print("Respuesta cruda:", resp)
try:
    print("Como texto:", resp.decode('utf-8', errors='ignore'))
except:
    pass

# si no hay respuesta, lee durante 1 segundo esperando bytes
if not resp:
    deadline = time.time() + 1.0
    collected = b''
    while time.time() < deadline:
        n = ser.in_waiting
        if n:
            collected += ser.read(n)
        time.sleep(0.05)
    print("Lectura acumulada:", collected.decode('utf-8', errors='ignore'))

ser.close()


Respuesta cruda: b''
Como texto: 
Lectura acumulada: 


# Test correcto para APC220

**Importante**: El APC220 NO usa comandos AT. Usa comandos específicos como `RD` (read config).

**Problemas comunes**:
- Respuesta vacía = módulo no responde (conexiones mal, SET no en LOW, baudrate incorrecto, o módulo apagado)
- Verificar que RX/TX no estén cruzados incorrectamente
- SET debe estar en LOW (0V) para modo CONFIG

In [13]:
import serial
import time

# Probando todos los baudrates con el comando correcto RD
BAUDRATES = [1200, 2400, 4800, 9600, 19200, 38400, 57600, 115200]

print("="*50)
print("TEST APC220 - Comando RD (Read Config)")
print("="*50)

for baud in BAUDRATES:
    print(f"\n🔍 Probando baudrate: {baud}")
    
    try:
        ser = serial.Serial(DEVICE, baudrate=baud, timeout=1)
        
        # Configurar líneas de control para modo CONFIG
        # RTS = True significa SET = LOW (modo CONFIG)
        ser.setRTS(True)   # SET = LOW (modo CONFIG)
        ser.setDTR(False)  # EN = HIGH (módulo encendido)
        time.sleep(0.5)    # Esperar a que entre en modo CONFIG
        
        # Limpiar buffers
        ser.reset_input_buffer()
        ser.reset_output_buffer()
        
        # Enviar comando RD (sin \r\n según el código del ESP32)
        comando = "RD\r\n"  # Probaremos con y sin line endings
        ser.write(comando.encode('utf-8'))
        ser.flush()
        print(f"  📤 Enviado: {comando.strip()}")
        
        # Esperar respuesta
        time.sleep(0.5)
        
        # Leer respuesta
        if ser.in_waiting > 0:
            response = ser.read(ser.in_waiting)
            print(f"  ✅ RESPUESTA RECIBIDA!")
            print(f"  📥 Bytes: {response}")
            print(f"  📄 Texto: {response.decode('utf-8', errors='ignore')}")
            print(f"  ⭐ BAUDRATE CORRECTO: {baud}")
            ser.close()
            break
        else:
            print(f"  ❌ Sin respuesta")
            
        ser.close()
        
    except Exception as e:
        print(f"  ⚠️ Error: {e}")

print("\n" + "="*50)

TEST APC220 - Comando RD (Read Config)

🔍 Probando baudrate: 1200
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 2400
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 4800
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 9600
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 19200
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 38400
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 57600
  📤 Enviado: RD
  ❌ Sin respuesta

🔍 Probando baudrate: 115200
  📤 Enviado: RD
  ❌ Sin respuesta



## Verificación de conexiones físicas

Antes de ejecutar el código, verifica:

1. **Alimentación**: El APC220 debe tener 5V y GND conectados
2. **Conexiones cruzadas**:
   - TX del adaptador → RX del APC220
   - RX del adaptador → TX del APC220
3. **Pin SET**: Debe estar conectado a una línea de control (RTS en el adaptador USB-Serial)
4. **Voltajes**: Si el adaptador es 3.3V y el APC220 es 5V, puede haber problemas de compatibilidad

# Test de Comunicación Loopback USB-Serial

**Objetivo**: Verificar que los dos adaptadores USB-Serial funcionan correctamente antes de conectar el APC220.

**Conexiones necesarias**:
- TX del puerto 1 → RX del puerto 2
- RX del puerto 1 → TX del puerto 2
- GND común entre ambos adaptadores

**Puertos disponibles**:
- Puerto 1: `/dev/cu.usbserial-010DC655`
- Puerto 2: `/dev/cu.usbserial-01657030`

In [32]:
import serial
import time
import threading

# Configuración de puertos
PORT_1 = "/dev/cu.usbserial-010DC655"  # Puerto transmisor
PORT_2 = "/dev/cu.usbserial-01657030"  # Puerto receptor
BAUDRATE = 9600

# Variable para controlar el hilo
running = True
mensajes_recibidos = []

def receptor():
    """Función que recibe mensajes en el puerto 2"""
    global running, mensajes_recibidos
    try:
        ser2 = serial.Serial(PORT_2, BAUDRATE, timeout=0.5)
        print(f"✅ Receptor iniciado en {PORT_2}")
        
        while running:
            if ser2.in_waiting > 0:
                data = ser2.readline().decode('utf-8', errors='ignore').strip()
                if data:
                    timestamp = time.strftime("%H:%M:%S")
                    mensaje = f"[{timestamp}] 📥 Recibido: {data}"
                    print(mensaje)
                    mensajes_recibidos.append(mensaje)
            time.sleep(0.1)
        
        ser2.close()
        print("🛑 Receptor detenido")
    except Exception as e:
        print(f"❌ Error en receptor: {e}")

# Iniciar hilo receptor
hilo_receptor = threading.Thread(target=receptor, daemon=True)
hilo_receptor.start()
time.sleep(1)  # Dar tiempo al receptor para iniciar

# Transmisor (en el hilo principal)
try:
    ser1 = serial.Serial(PORT_1, BAUDRATE, timeout=1)
    print(f"✅ Transmisor iniciado en {PORT_1}")
    print("="*60)
    print("📤 Enviando mensajes cada 2 segundos...")
    print("   Presiona Ctrl+C para detener o espera 20 segundos")
    print("="*60)
    
    contador = 1
    for i in range(10):  # Enviar 10 mensajes
        if not running:
            break
            
        mensaje = f"Test mensaje #{contador} desde {PORT_1}\n"
        ser1.write(mensaje.encode('utf-8'))
        ser1.flush()
        
        timestamp = time.strftime("%H:%M:%S")
        print(f"[{timestamp}] 📤 Enviado: Test mensaje #{contador}")
        
        contador += 1
        time.sleep(2)
    
    # Detener todo
    running = False
    time.sleep(1)
    ser1.close()
    print("\n" + "="*60)
    print("🛑 Transmisor detenido")
    print("="*60)
    
    # Resumen
    print(f"\n📊 RESUMEN:")
    print(f"   Mensajes enviados: {contador - 1}")
    print(f"   Mensajes recibidos: {len(mensajes_recibidos)}")
    
    if len(mensajes_recibidos) == contador - 1:
        print("   ✅ COMUNICACIÓN EXITOSA - Todos los mensajes fueron recibidos")
    elif len(mensajes_recibidos) > 0:
        print("   ⚠️ COMUNICACIÓN PARCIAL - Algunos mensajes se recibieron")
    else:
        print("   ❌ SIN COMUNICACIÓN - No se recibieron mensajes")
        print("\n   Verifica:")
        print("   - TX del puerto 1 conectado a RX del puerto 2")
        print("   - RX del puerto 1 conectado a TX del puerto 2")
        print("   - GND común entre ambos adaptadores")
        print("   - Baudrate correcto en ambos puertos")
    
except KeyboardInterrupt:
    print("\n⚠️ Interrumpido por el usuario")
    running = False
    time.sleep(1)
    ser1.close()
except Exception as e:
    print(f"❌ Error en transmisor: {e}")
    running = False

✅ Receptor iniciado en /dev/cu.usbserial-01657030
✅ Transmisor iniciado en /dev/cu.usbserial-010DC655
📤 Enviando mensajes cada 2 segundos...
   Presiona Ctrl+C para detener o espera 20 segundos
[19:52:03] 📤 Enviado: Test mensaje #1
[19:52:03] 📥 Recibido: Test mensaje #1 desde /dev/cu.usbserial-010DC655
[19:52:05] 📤 Enviado: Test mensaje #2
[19:52:05] 📥 Recibido: Test mensaje #2 desde /dev/cu.usbserial-010DC655
[19:52:07] 📤 Enviado: Test mensaje #3
[19:52:07] 📥 Recibido: Test mensaje #3 desde /dev/cu.usbserial-010DC655
[19:52:09] 📤 Enviado: Test mensaje #4
[19:52:09] 📥 Recibido: Test mensaje #4 desde /dev/cu.usbserial-010DC655
[19:52:11] 📤 Enviado: Test mensaje #5
[19:52:11] 📥 Recibido: Test mensaje #5 desde /dev/cu.usbserial-010DC655
[19:52:13] 📤 Enviado: Test mensaje #6
[19:52:13] 📥 Recibido: Test mensaje #6 desde /dev/cu.usbserial-010DC655
[19:52:15] 📤 Enviado: Test mensaje #7
[19:52:15] 📥 Recibido: Test mensaje #7 desde /dev/cu.usbserial-010DC655
[19:52:17] 📤 Enviado: Test mensaje #

## Interpretación de resultados

**✅ COMUNICACIÓN EXITOSA**: 
- Ambos adaptadores funcionan correctamente
- Las conexiones TX/RX están bien
- Puedes proceder a conectar el APC220

**⚠️ COMUNICACIÓN PARCIAL**:
- Los adaptadores funcionan pero hay pérdida de datos
- Revisa las conexiones físicas (pueden estar flojas)
- Prueba con un baudrate más bajo (ej: 4800)

**❌ SIN COMUNICACIÓN**:
- Verifica que los cables estén conectados correctamente
- TX de un puerto debe ir a RX del otro (y viceversa)
- Asegúrate de que GND esté conectado entre ambos
- Verifica que los adaptadores tengan alimentación

# Monitor Serial Continuo

**Objetivo**: Escuchar continuamente un puerto serial y mostrar todos los mensajes que lleguen en tiempo real.

**Instrucciones**:
1. Configura el puerto y baudrate que quieres monitorear
2. Ejecuta la celda
3. Todos los mensajes recibidos se mostrarán en pantalla
4. Presiona el botón ⏹️ (Stop) para detener el monitor

In [31]:
import serial
import time

# ============ CONFIGURACIÓN ============
PUERTO = "/dev/cu.usbserial-010DC655"  # Cambia esto al puerto que quieres monitorear
BAUDRATE = 9600                         # Cambia esto al baudrate correcto
# =======================================

print("="*70)
print(f"🔍 MONITOR SERIAL CONTINUO")
print(f"   Puerto: {PUERTO}")
print(f"   Baudrate: {BAUDRATE}")
print("="*70)
print("📡 Escuchando mensajes... (Presiona ⏹️ Stop para detener)")
print("="*70)

try:
    ser = serial.Serial(PUERTO, BAUDRATE, timeout=0.5)
    time.sleep(0.5)  # Esperar inicialización
    
    contador_mensajes = 0
    
    # Loop infinito - se detendrá al presionar el botón Stop
    while True:
        # Leer línea por línea
        if ser.in_waiting > 0:
            try:
                # Leer una línea completa
                linea = ser.readline().decode('utf-8', errors='ignore').strip()
                
                if linea:  # Solo mostrar si hay contenido
                    contador_mensajes += 1
                    timestamp = time.strftime("%H:%M:%S")
                    print(f"[{timestamp}] #{contador_mensajes}: {linea}")
                    
            except Exception as e:
                print(f"⚠️ Error al leer: {e}")
        
        time.sleep(0.01)  # Pequeña pausa para no saturar el CPU
        
except KeyboardInterrupt:
    print("\n" + "="*70)
    print("🛑 Monitor detenido por el usuario")
    print(f"📊 Total de mensajes recibidos: {contador_mensajes}")
    print("="*70)
    ser.close()
    
except Exception as e:
    print(f"\n❌ Error: {e}")
    print("Verifica que el puerto y baudrate sean correctos")
    
finally:
    if 'ser' in locals() and ser.is_open:
        ser.close()

🔍 MONITOR SERIAL CONTINUO
   Puerto: /dev/cu.usbserial-010DC655
   Baudrate: 9600
📡 Escuchando mensajes... (Presiona ⏹️ Stop para detener)

🛑 Monitor detenido por el usuario
📊 Total de mensajes recibidos: 0


# Emisor continuo por USB

Envia mensajes cada 2 segundos desde `/dev/cu.usbserial-01657030` para probar recepción en el otro adaptador / APC220.

In [69]:
import serial, time

PORT = "/dev/cu.usbserial-01657030"
BAUD = 9600
INTERVAL_MS = 2000

print("="*60)
print("🚀 Emisor continuo")
print(f"Puerto: {PORT}")
print(f"Baudrate: {BAUD}")
print("Enviando cada 2s... (Ctrl+C para parar)")
print("="*60)

try:
    ser = serial.Serial(PORT, BAUD, timeout=1)
    time.sleep(0.5)
    
    counter = 1
    while True:
        msg = f"PING #{counter} desde {PORT}\n"
        ser.write(msg.encode('utf-8'))
        ser.flush()
        ts = time.strftime('%H:%M:%S')
        print(f"[{ts}] enviado: {msg.strip()}")
        counter += 1
        time.sleep(INTERVAL_MS/1000.0)

except KeyboardInterrupt:
    print("\n🛑 Emisión detenida por el usuario")
except Exception as e:
    print(f"❌ Error: {e}")
finally:
    try:
        ser.close()
    except:
        pass

🚀 Emisor continuo
Puerto: /dev/cu.usbserial-01657030
Baudrate: 9600
Enviando cada 2s... (Ctrl+C para parar)
[20:03:22] enviado: PING #1 desde /dev/cu.usbserial-01657030
[20:03:24] enviado: PING #2 desde /dev/cu.usbserial-01657030
[20:03:26] enviado: PING #3 desde /dev/cu.usbserial-01657030
[20:03:28] enviado: PING #4 desde /dev/cu.usbserial-01657030
[20:03:30] enviado: PING #5 desde /dev/cu.usbserial-01657030
[20:03:32] enviado: PING #6 desde /dev/cu.usbserial-01657030
[20:03:34] enviado: PING #7 desde /dev/cu.usbserial-01657030
[20:03:36] enviado: PING #8 desde /dev/cu.usbserial-01657030
❌ Error: write failed: [Errno 6] Device not configured
